In [253]:
import numpy as np

In [254]:
import pandas as pd

In [255]:
import nltk

In [256]:
import sklearn

In [257]:

cols = ['polarity','id','date','flag','user','text']
set_encoding = "ISO-8859-1"


In [258]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding=set_encoding, names=cols)
#df.sample(10)

In [259]:
np.sum(df.isnull().any(axis=1))

0

In [286]:
data=df[['text','polarity']]
data.sample(10)

,text,polarity
311015,as expected... errors,0
463251,exam went gr8.... missin my friends,0
386423,Pussy cat has been missing for 3 days now. Fea...,0
1045643,@cappo goodnight,4
525371,@_CrC_ hells ya! But they never respond to me,0
1462914,Watching F1 Turkey GP at home with Beers&amp;C...,4
1207166,ok that's was freaken awsome,4
25969,i want a little pet so bad,0
437090,@trythis__ I envy you,0
1293202,"@DaivRawks Yeah, I sure wasted a lot of time ...",4


In [261]:
data['polarity'] = data['polarity'].replace(4,1)


/var/folders/wt/vkwnpm091dqdlm0xqwfjxq_80000gn/T/ipykernel_56187/335142359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['polarity'] = data['polarity'].replace(4,1)


In [262]:
data_pos = data[data['polarity'] == 1].iloc[:int(20000)]
data_neg = data[data['polarity'] == 0].iloc[:int(20000)]
dataset = pd.concat([data_pos, data_neg])
dataset['text']=dataset['text'].str.lower()
dataset['text'].tail()
#dataset.sample(20)

19995    not much time off this weekend, work trip to m...
19996                            one more day of holidays 
19997    feeling so down right now .. i hate you damn h...
19998    geez,i hv to read the whole book of personalit...
19999    i threw my sign at donnie and he bent over to ...
Name: text, dtype: object

In [263]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))
#print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mikebazarsky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [264]:
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


In [265]:
dataset['text'] = dataset['text'].apply(lambda text: cleaning_stopwords(text))
dataset['text'].tail()

19995    much time weekend, work trip malmï¿½ fri-sat t...
19996                                     one day holidays
19997                   feeling right .. hate damn humprey
19998    geez,i hv read whole book personality types em...
19999     threw sign donnie bent get thingee made sad face
Name: text, dtype: object

In [266]:
import string
english_punctuations = string.punctuation
print(english_punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [267]:
#Removing punctuation
def cleaning_punctuations(text):
    translator = str.maketrans('', '', english_punctuations)
    return text.translate(translator)
dataset['text']= dataset['text'].apply(lambda x: cleaning_punctuations(x))
dataset['text'].tail()

19995    much time weekend work trip malmï¿½ frisat tod...
19996                                     one day holidays
19997                     feeling right  hate damn humprey
19998    geezi hv read whole book personality types emb...
19999     threw sign donnie bent get thingee made sad face
Name: text, dtype: object

In [268]:
#Removing URLs
import re
dataset['text'] = dataset['text'].astype(str)
def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)

dataset['text'] = dataset['text'].apply(lambda x: cleaning_URLs(x))
dataset['text'].head()

800000                   love health4uandpets u guys r best
800001    im meeting one besties tonight cant wait  girl...
800002    darealsunisakim thanks twitter add sunisa got ...
800003    sick really cheap hurts much eat real food plu...
800004                       lovesbrooklyn2 effect everyone
Name: text, dtype: object

In [269]:
#Removing numbers
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_numbers(x))
dataset['text'].head()

800000                    love healthuandpets u guys r best
800001    im meeting one besties tonight cant wait  girl...
800002    darealsunisakim thanks twitter add sunisa got ...
800003    sick really cheap hurts much eat real food plu...
800004                        lovesbrooklyn effect everyone
Name: text, dtype: object

In [270]:
#Tokenizing
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

In [271]:
tokenizer = RegexpTokenizer(r'\w+')
dataset['text'] = dataset['text'].apply(tokenizer.tokenize)
dataset['text'].head()

800000             [love, healthuandpets, u, guys, r, best]
800001    [im, meeting, one, besties, tonight, cant, wai...
800002    [darealsunisakim, thanks, twitter, add, sunisa...
800003    [sick, really, cheap, hurts, much, eat, real, ...
800004                    [lovesbrooklyn, effect, everyone]
Name: text, dtype: object

In [272]:
#Stemming
import nltk
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
dataset['text']= dataset['text'].apply(lambda x: stemming_on_text(x))
dataset['text'].head()


800000             [love, healthuandpets, u, guys, r, best]
800001    [im, meeting, one, besties, tonight, cant, wai...
800002    [darealsunisakim, thanks, twitter, add, sunisa...
800003    [sick, really, cheap, hurts, much, eat, real, ...
800004                    [lovesbrooklyn, effect, everyone]
Name: text, dtype: object

In [273]:
#Lemmatizer
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data
dataset['text'] = dataset['text'].apply(lambda x: lemmatizer_on_text(x))
dataset['text'].head()

800000             [love, healthuandpets, u, guys, r, best]
800001    [im, meeting, one, besties, tonight, cant, wai...
800002    [darealsunisakim, thanks, twitter, add, sunisa...
800003    [sick, really, cheap, hurts, much, eat, real, ...
800004                    [lovesbrooklyn, effect, everyone]
Name: text, dtype: object

In [274]:
#Separating input feature and label
X=data.text
y=data.polarity

In [275]:
from sklearn.model_selection import train_test_split


In [276]:
#Splitting data into Train and Test subset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.05, random_state =935)


In [277]:
#Fit tf-dif vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectoriser.fit(X_train)

KeyboardInterrupt: 

In [ ]:
#Transform tf-dif vectorizer
X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
def model_Evaluate(model):
    # Predict values for Test dataset
    y_pred = model.predict(X_test)
    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
BNBmodel = BernoulliNB()
BNBmodel.fit(X_train, y_train)
model_Evaluate(BNBmodel)
y_pred1 = BNBmodel.predict(X_test)

              precision    recall  f1-score   support

           0       0.81      0.79      0.80     39983
           1       0.80      0.81      0.80     40017

    accuracy                           0.80     80000
   macro avg       0.80      0.80      0.80     80000
weighted avg       0.80      0.80      0.80     80000



In [ ]:
SVCmodel = LinearSVC()
SVCmodel.fit(X_train, y_train)
model_Evaluate(SVCmodel)
y_pred2 = SVCmodel.predict(X_test)

              precision    recall  f1-score   support

           0       0.82      0.81      0.82     39983
           1       0.81      0.82      0.82     40017

    accuracy                           0.82     80000
   macro avg       0.82      0.82      0.82     80000
weighted avg       0.82      0.82      0.82     80000



In [ ]:
LRmodel = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
LRmodel.fit(X_train, y_train)
model_Evaluate(LRmodel)
y_pred3 = LRmodel.predict(X_test)

              precision    recall  f1-score   support

           0       0.83      0.82      0.83     39983
           1       0.82      0.83      0.83     40017

    accuracy                           0.83     80000
   macro avg       0.83      0.83      0.83     80000
weighted avg       0.83      0.83      0.83     80000



In [ ]:
import requests

In [ ]:
token = 'pk_62e14354ce4f41b5bbb10e69f666afb1'
#https://cloud.iexapis.com/stable/tops?token=YOUR_TOKEN_HERE&symbols=aapl

r = requests.get('https://cloud.iexapis.com/stable/tops?token={}&symbols=aapl'.format(token))
print(r)
print(r.content)

<Response [200]>
b'[]'


In [280]:
import json

In [345]:
#HTTPS request
r = requests.get('https://cloud.iexapis.com/stable/ref-data/iex/symbols?token=pk_62e14354ce4f41b5bbb10e69f666afb1')

In [346]:
#Get the names of all the stocks in the list
stockNames = []
for stock in r.json():
    stockNames.append(stock['symbol'])
#print(stockNames)

['A', 'AA', 'AAA', 'AAAU', 'AAC', 'AAC+', 'AAC=', 'AACG', 'AACI', 'AACIU', 'AACIW', 'AADI', 'AADR', 'AAIC', 'AAIC-B', 'AAIC-C', 'AAIN', 'AAL', 'AAM-A', 'AAM-B', 'AAMC', 'AAME', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAQC', 'AAQC+', 'AAQC=', 'AAT', 'AATC', 'AAU', 'AAWW', 'AAXJ', 'AB', 'ABB', 'ABBV', 'ABC', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABEQ', 'ABEV', 'ABG', 'ABGI', 'ABIO', 'ABM', 'ABMD', 'ABNB', 'ABOS', 'ABR', 'ABR-D', 'ABR-E', 'ABR-F', 'ABSI', 'ABST', 'ABT', 'ABTX', 'ABUS', 'ABVC', 'AC', 'ACA', 'ACAB', 'ACABU', 'ACABW', 'ACAD', 'ACAH', 'ACAHU', 'ACAHW', 'ACAQ', 'ACAQ+', 'ACAQ=', 'ACAX', 'ACAXR', 'ACAXU', 'ACAXW', 'ACB', 'ACBA', 'ACBAU', 'ACBAW', 'ACC', 'ACCD', 'ACCO', 'ACDI', 'ACDI+', 'ACDI=', 'ACEL', 'ACER', 'ACES', 'ACET', 'ACEV', 'ACEVU', 'ACEVW', 'ACGL', 'ACGLN', 'ACGLO', 'ACH', 'ACHC', 'ACHL', 'ACHR', 'ACHR+', 'ACHV', 'ACI', 'ACII', 'ACII+', 'ACII=', 'ACIO', 'ACIU', 'ACIW', 'ACKIT', 'ACKIU', 'ACKIW', 'ACLS', 'ACLX', 'ACM', 'ACMR', 'ACN', 'ACNB', 'ACOR', 'ACP', 'ACP-A', 'ACQR', 

In [342]:
import random
chosenStocks = random.sample(stockNames, 5)
print(chosenStocks)

['MLCO', 'NPFD', 'TX', 'NDAQ', 'AVAC']
